<a href="https://colab.research.google.com/github/aman-24052001/RAG-Learning/blob/main/Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install openai pinecone-client datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling request

In [3]:
# Import necessary libraries
import openai
import pinecone
from datasets import load_dataset
import numpy as np
import pandas as pd

In [6]:
pip install pinecone-client

In [8]:
from pinecone import Pinecone, ServerlessSpec

In [13]:
# Initialize Pinecone
pinecone = Pinecone(api_key="YOUR_PINECONE_API_KEY")

In [16]:
# Define the environment (cloud and region)
cloud = 'aws'
region = 'us-east-1'

# Create Pinecone index
index_name = "qa-bot"
if index_name not in [index.name for index in pinecone.list_indexes()]:
    pinecone.create_index(
        name="quickstart",
        dimension=8,  # Assuming 8 dimensions for embeddings
        metric='cosine',  # You can use 'cosine', 'euclidean', or 'dotproduct'
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
index = pinecone.Index(index_name)

In [17]:
# Initialize OpenAI
openai.api_key = "YOUR_OPENAI_API_KEY"

In [18]:
# Load a dataset (e.g., a Q&A dataset from Hugging Face)
dataset = load_dataset("squad", split="train[:1000]")  # Load a small sample for demo

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
# Function to generate embeddings using OpenAI API
def get_embedding(text, model="text-embedding-ada-002"):
    response = openai.Embedding.create(input=text, model=model)
    return response['data'][0]['embedding']

In [ ]:
# Index the dataset in Pinecone
for i, example in enumerate(dataset):
    question = example["question"]
    context = example["context"]
    embedding = get_embedding(question + " " + context)
    index.upsert([(str(i), embedding)])

In [20]:
# Function to retrieve similar contexts
def retrieve_context(query, top_k=5):
    query_embedding = get_embedding(query)
    result = index.query(query_embedding, top_k=top_k)
    return [dataset[int(match["id"])]["context"] for match in result["matches"]]


In [21]:
# Function to generate answer using OpenAI's GPT model
def generate_answer(query, context, model="text-davinci-003"):
    prompt = f"Question: {query}\n\nContext: {context}\n\nAnswer:"
    response = openai.Completion.create(engine=model, prompt=prompt, max_tokens=150)
    return response['choices'][0]['text'].strip()


In [ ]:
# Example usage
query = "What is the capital of India?"
contexts = retrieve_context(query)
context = " ".join(contexts)
answer = generate_answer(query, context)
print("Query:", query)
print("Answer:", answer)